In [1]:
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
          super(ResidualBlock, self).__init__()
          self.conv1 = nn.Sequential(
              nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU())
          self.conv2 = nn.Sequential(
              nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
              nn.BatchNorm2d(out_channels))
          self.downsample = downsample
          self.relu = nn.ReLU()
          self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [3]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super (ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),

            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [4]:
class givemodel():
    def __init__(self, train_path, test_path, learning_rate, num_epochs):
        self.train_path = train_path
        self.test_path = test_path
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.transforms = transforms.Compose([
                        transforms.Resize((224,224)),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
                        transforms.Normalize(mean =[0.6953, 0.6752, 0.6424],
                            std=[0.1198, 0.1166, 0.1154],) #Determine mean and std of the dataset using Image_Normalization.ipynb
                    ])

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        train_loader = DataLoader(
            torchvision.datasets.ImageFolder(train_path, transform=self.transforms),
            batch_size=64, shuffle=True
        )

        test_loader = DataLoader(
            torchvision.datasets.ImageFolder(test_path, transform=self.transforms),
            batch_size=32, shuffle=True)

        self.model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(self.device)
        self.optimizer = Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=0.0001)
        self.loss_function = nn.CrossEntropyLoss()
        self.num_epochs = self.num_epochs

        train_count = len(glob.glob(self.train_path + '/**/*.jpg')) #Change image format according to dataset
        test_count = len(glob.glob(self.test_path + '/**/*.jpg'))

        print(train_count, test_count)

        best_accuracy = 0.0

        for epoch in range(num_epochs):
              # Evaluation and training on training dataset
              self.model.train()
              train_accuracy = 0.0
              train_loss = 0.0
              for i, (images, labels) in enumerate (train_loader):
                    if torch.cuda.is_available():
                        images = Variable(images.cuda())
                        labels = Variable(labels.cuda())
                    self.optimizer.zero_grad()
                    outputs = self.model(images)
                    loss = self.loss_function(outputs, labels)
                    loss.backward()
                    self.optimizer.step()
                    train_loss += loss.cpu().data * images.size(0)
                    _, prediction = torch.max(outputs.data, 1)

                    train_accuracy += int(torch.sum(prediction == labels.data))

              train_accuracy = train_accuracy / train_count
              train_loss = train_loss / train_count

              # Evaluation on testing dataset
              self.model.eval()
              test_accuracy = 0.0
              for i, (images, labels) in enumerate (test_loader):
                  if torch.cuda.is_available():
                      images = Variable(images.cuda())
                      labels = Variable(labels.cuda())
                  outputs = self.model(images)
                  _, prediction = torch.max(outputs.data, 1)
                  test_accuracy += int(torch.sum(prediction == labels.data))
              test_accuracy = test_accuracy/test_count
              print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) + ' Train Accuracy: ' + str(
                  train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))

              # Save the best model
              if test_accuracy > best_accuracy:
                  torch.save(self.model.state_dict(), 'best_checkpoint1.model')
                  best_accuracy = test_accuracy

    def save_model(self, name_of_model):
        torch.save(self.model, name_of_model)

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arunrk7/surface-crack-detection")
print("Path to dataset files:", path)

100%|██████████| 233M/233M [00:01<00:00, 124MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/arunrk7/surface-crack-detection/versions/1


In [6]:
!ls /root/.cache/kagglehub/datasets/arunrk7/surface-crack-detection/versions/1

Negative  Positive


In [7]:
import os
import shutil
import random

# Paths to the downloaded dataset and class folders
source_dir = '/root/.cache/kagglehub/datasets/arunrk7/surface-crack-detection/versions/1'
negative_dir = os.path.join(source_dir, 'Negative')
positive_dir = os.path.join(source_dir, 'Positive')

# Define paths for train and test directories
train_dir = '/content/dataset/train'
test_dir = '/content/dataset/test'
train_negative_dir = os.path.join(train_dir, 'Negative')
train_positive_dir = os.path.join(train_dir, 'Positive')
test_negative_dir = os.path.join(test_dir, 'Negative')
test_positive_dir = os.path.join(test_dir, 'Positive')

# Create train and test subdirectories
os.makedirs(train_negative_dir, exist_ok=True)
os.makedirs(train_positive_dir, exist_ok=True)
os.makedirs(test_negative_dir, exist_ok=True)
os.makedirs(test_positive_dir, exist_ok=True)

# Function to split and copy files
def split_and_copy_files(class_dir, train_dest, test_dest, split_ratio=0.7):
    all_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    random.shuffle(all_files)  # Shuffle the files
    split_index = int(split_ratio * len(all_files))  # Calculate split index

    train_files = all_files[:split_index]
    test_files = all_files[split_index:]

    # Copy to train and test directories
    for f in train_files:
        shutil.copy(os.path.join(class_dir, f), os.path.join(train_dest, f))
    for f in test_files:
        shutil.copy(os.path.join(class_dir, f), os.path.join(test_dest, f))

    return len(train_files), len(test_files)

# Split Negative class
train_neg_count, test_neg_count = split_and_copy_files(negative_dir, train_negative_dir, test_negative_dir)

# Split Positive class
train_pos_count, test_pos_count = split_and_copy_files(positive_dir, train_positive_dir, test_positive_dir)

print(f"Negative - Train: {train_neg_count}, Test: {test_neg_count}")
print(f"Positive - Train: {train_pos_count}, Test: {test_pos_count}")
print(f"Total images in Train: {train_neg_count + train_pos_count}, Total images in Test: {test_neg_count + test_pos_count}")

Negative - Train: 14000, Test: 6000
Positive - Train: 14000, Test: 6000
Total images in Train: 28000, Total images in Test: 12000


In [8]:
if __name__ == '__main__':
      train_path= train_dir
      test_path= test_dir
      obj = givemodel(train_path, test_path, 0.001, 2)
      obj.save_model("Custom_ResNet34_Model")

28000 12000
Epoch: 0 Train Loss: tensor(0.0515) Train Accuracy: 0.9856428571428572 Test Accuracy: 0.9685833333333334
Epoch: 1 Train Loss: tensor(0.0234) Train Accuracy: 0.9935357142857143 Test Accuracy: 0.978
